In [ ]:
import timeit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error

## may not do pipeline, since we do not do encoding and imputation;
## only transfor and fit

# load and explore dataset

In [ ]:
##import training data and test data (to be predicted)
train = pd.read_csv('training_data.csv')
test = pd.read_csv('validation_data.csv')

In [ ]:
##data dimensions and summary statistics
print(train.shape)
print(test.shape)
print(train.columns)
train.describe()

In [ ]:
##show first five rows of data
train.head()

In [ ]:
##check how many unique sites
print("unique sites of training set: ",train['Location ID'].unique())
print(len(train['Location ID'].unique()))

print('unique sites of test set: ',test['Location ID'].unique())
len(test['Location ID'].unique())

In [ ]:
# ##cannot do train-test-split, regression along depth for each site
# print("number of training set that goes for validation: ",len(train_full['Location ID'].unique())*0.3)

# ##so randomly choose three sites as candidates for validation set
# valid_id = ['EL','CB','AV']
# train = train_full[~train_full['Location ID'].isin(valid_id)]
# valid = train_full[train_full['Location ID'].isin(valid_id)]
# print("shape of train: ",train.shape)
# print('shape of valid: ',valid.shape)

## no need to set three additional sites for validation since we will use cross validation to test model performance 

# EDA

In [ ]:
##plot cone tip resistance,normalized ENTRHU, and blowcount for each site
##every three sites on the same graph
color_list = ['r','b','g','c','m','y','k']

k = 0
while k < train_full['Location ID'].unique().shape[0]:
    ##plot every three sites
    site_id = train_full['Location ID'].unique()[k:(k+3)]
    ##initiate graph setting
    fig,((ax1,ax2,ax3,ax4)) = plt.subplots(1,4,sharey = True,figsize = (20,9))
    i = 0
    for location in site_id:
        locationdata = train_full[train_full['Location ID']==location].copy()
        ax1.plot(locationdata['qc [MPa]'],locationdata['z [m]'],color = color_list[i],label = location)
        ax2.plot(locationdata['fs [MPa]'],locationdata['z [m]'],color = color_list[i],label = location)
        ax3.plot(locationdata['Normalised ENTRHU [-]'],locationdata['z [m]'],color = color_list[i],label = location)
        ax4.plot(locationdata['Blowcount [Blows/m]'],locationdata['z [m]'],color = color_list[i],label = location)
        i+=1
    for ax in (ax1,ax2,ax3,ax4):
        ax.xaxis.tick_top()
        ax.xaxis.set_label_position("top")
        ax.grid()
        ax.set_ylim(50,0)
        ax.tick_params(axis = 'x',labelsize = 14)
        ax.legend(loc = 'lower right',prop = {'size':14})
    ax1.set_xlabel(r"Cone tip resistance [MPa]",fontsize = 16)
    ax2.set_xlabel(r"Sleeve friction [MPa]",fontsize = 16)
    ax3.set_xlabel(r"Normalized ENTRHU (-)",fontsize = 16)
    ax4.set_xlabel(r"Blowcount (blows/m)",fontsize = 16)
    ax1.tick_params(axis = 'y',labelsize = 14)
    ax1.set_xlim(0,120)
    ax2.set_xlim(0,2.5)
    ax3.set_xlim(0,1)
    ax4.set_xlim(0,200)
    plt.show()
    k+=3

In [ ]:
##plot pore water pressure u2, normalized hammer energy, number of blows, and pile penetration
k = 0
while k < train_full['Location ID'].unique().shape[0]:
    site_id = train_full['Location ID'].unique()[k:(k+3)]
    fig,((ax1,ax2,ax3,ax4)) = plt.subplots(1,4,sharey = True,figsize = (20,9))
    i = 0
    for location in site_id:
        locationdata = train_full[train_full['Location ID']==location].copy()
        ax1.plot(locationdata['u2 [MPa]'],locationdata['z [m]'],color = color_list[i],label = location)
        ax2.plot(locationdata['Normalised hammer energy [-]'],locationdata['z [m]'],color = color_list[i],label = location)
        ax3.plot(locationdata['Number of blows'],locationdata['z [m]'],color = color_list[i],label = location)
        ax4.plot(locationdata['Pile penetration [m]'],locationdata['z [m]'],color = color_list[i],label = location)
        i+=1
    for ax in (ax1,ax2,ax3,ax4):
        ax.xaxis.tick_top()
        ax.xaxis.set_label_position('top')
        ax.grid()
        ax.set_ylim(50,0)
        ax.tick_params(axis = 'x',labelsize = 14)
        ax.legend(loc = 'lower right',prop = {'size':14})
    ax1.set_xlabel(r"Pore water pressure (MPa)",fontsize = 16)
    ax2.set_xlabel(r"Normalised hammer energy (-)",fontsize = 16)
    ax3.set_xlabel(r"Number of blows",fontsize = 16)
    ax4.set_xlabel(r"Pile penetration (m)",fontsize = 16)
    ax1.set_ylabel(r"Depth below mudline (m)",fontsize = 16)
    ax1.tick_params(axis = 'y',labelsize = 14)
    ax1.set_xlim(-0.5,2)
    ax2.set_xlim(0,1)
    ax3.set_xlim(0,2500)
    ax4.set_xlim(0,50)
    plt.show()
    k+=3

# cross validation to determine best alpha-value to optimize the linear coefficients

# create new features
# use IC-05 method (Jardine)

In [ ]:
##first define input parameters
##atmospheric pressure (MPa)
P_atm = 0.101325
##define interface friction angle between sand and pile
tan_delta = np.tan(29)
##unit of D_CPT is meter
D_CPT = 0.035814

In [ ]:
train_unit = pd.DataFrame()
train_unit = train.copy()
##compute the h-part
train_unit['h [m]'] = train['Pile penetration [m]']-train['z [m]']
##compute the effective area of the pipe pile
train_unit['R_star [m]'] = pow(pow(train["Diameter [m]"],2)/4-
                          pow(train["Diameter [m]"]-2*train["Bottom wall thickness [mm]"]/1000,2)/4,0.5)
##compute max function which involves h-part
train_unit['h_R_max'] =  np.maximum(train_unit["h [m]"]/train_unit["R_star [m]"],np.full(train.shape[0],8))
##compute the unit shaft friction
train_unit['tao [MPa]'] = 0.9*0.029*1.0*train["qc [MPa]"]*pow((10.2*train["z [m]"])/P_atm/1000,0.13)*pow(train_unit["h_R_max"],-0.38)*tan_delta
##compute the unit end bearing
train_unit['qb [MPa]'] = (0.5-0.25*np.log(train["Diameter [m]"]/D_CPT))*train["qc [MPa]"]

# cross validation parameters

In [ ]:
CV_fold = np.arange(1,6)
alpha_sen = np.linspace(1e-3,0.5,500)

##store the best MSE, R^2
MSE_train = np.zeros(len(alpha_sen))
R2_train = np.zeros(len(alpha_sen))

linearized_features = ['linearized ENTRHU','linearized Qult','linearized z']

In [ ]:
t_begin = timeit.default_timer()
for i,alpha in enumerate(alpha_sen):
    ##random group assignments for cross validation (unbalanced design)
    index_values = np.random.choice(CV_fold,train['Location ID'].unique().shape[0],replace = True)
    MSE_temp = np.zeros(len(CV_fold))
    R2_temp = np.zeros(len(CV_fold))
    for k in np.arange(1,len(CV_fold)+1):
        index_out = train_unit['Location ID'].unique()[np.where(index_values==k)]
        data_out = pd.DataFrame()
        data_in = pd.DataFrame()
        ##data_out to compute loss
        data_out = train_unit[train_unit['Location ID'].isin(index_out)].copy()
        ##data_out to fit teh data
        data_in = train_unit[~train_unit['Location ID'].isin(index_out)].copy()
        
        ##training set (left in)
        data_in['Rs [kN]'] = alpha*(np.pi*data_in["Diameter [m]"]*data_in["z [m]"].diff()*
                                          data_in["tao [MPa]"]).cumsum()
        data_in['Qb [kN]'] = (np.pi/4*pow(data_in["Diameter [m]"],2))*data_in["qb [MPa]"]
        data_in['Qult [kN]'] = data_in["Rs [kN]"]+data_in["Qb [kN]"]
        ##linearized features
        data_in["linearized ENTRHU"] = np.tanh(data_in["Normalised ENTRHU [-]"]*5-0.5)
        data_in["linearized Qult"] = np.tanh(data_in["Qult [kN]"]/1000-1)
        data_in["linearized z"] = np.tanh(data_in["z [m]"]/10-0.5)
    
        ##validation set (left out)
        data_out['Rs [kN]'] = alpha*(np.pi*data_out["Diameter [m]"]*
                                     data_out["z [m]"].diff()*data_out["tao [MPa]"]).cumsum()
        data_out['Qb [kN]'] = (np.pi/4*pow(data_out["Diameter [m]"],2))*data_out["qb [MPa]"]
        data_out['Qult [kN]'] = data_out["Rs [kN]"]+data_out["Qb [kN]"]
        ##linearized features
        data_out["linearized ENTRHU"] = np.tanh(data_out["Normalised ENTRHU [-]"]*5-0.5)
        data_out["linearized Qult"] = np.tanh(data_out["Qult [kN]"]/1000-1)
        data_out["linearized z"] = np.tanh(data_out["z [m]"]/10-0.5)
        
        ##drop na values
        x_out = data_out.dropna()[linearized_features]
        y_out = data_out.dropna()['Blowcount [Blows/m]']
        x_in = data_in.dropna()[linearized_features]
        y_in = data_in.dropna()['Blowcount [Blows/m]']
        
        ##fit a linear model
        lm1 = LinearRegression().fit(x_in,y_in)
        y_pred_lm1 = lm1.predict(x_out)
        
        MSE_temp[k-1] = mean_squared_error(y_out,y_pred_lm1)
        R2_temp[k-1] = r2_score(y_out,y_pred_lm1)
        
    ##calculate teh average MSE and R2
    MSE_train[i] = np.mean(MSE_temp)
    R2_train[i] = np.mean(R2_temp)
t_end = timeit.default_timer()
print("Run time is: ",t_end-t_begin)
print(MSE_train)
print(R2_train)

In [ ]:
print(min(MSE_train))
print(max(R2_train))

##find alpha values with maximum R2 adn minimum MSE
print(alpha_sen[np.argmin(MSE_train)])
print(alpha_sen[np.argmax(R2_train)])

print([np.argmin(MSE_train),np.argmax(R2_train)])

##plot the sensitivity analysis results
##MSE versus alpha values
plt.plot(alpha_sen,MSE_train)
plt.xlabel(r"alpha")
plt.ylabel(r"MSE")
plt.show()
##R2 versus alpha values
plt.plot(alpha_sen,R2_train)
plt.xlabel(r"alpha")
plt.ylabel(r"R^2")

##note: too much randomness to find the optimal alpha

# sensitivity analysis for alpha

In [ ]:
##train test split
##so randomly choose three sites as candidates for validation set
valid_id = ['EL','CB','AV']
train = train_full[~train_full['Location ID'].isin(valid_id)]
valid = train_full[train_full['Location ID'].isin(valid_id)]
print("shape of train: ",train.shape)
print('shape of valid: ',valid.shape)

In [ ]:
##first define input parameters
##atmospheric pressure (MPa)
P_atm = 0.101325
##define interface friction angle between sand and pile
tan_delta = np.tan(29)
##unit of D_CPT is meter
D_CPT = 0.035814

##input data for train set
train_unit = pd.DataFrame()
train_unit = train.copy()
##compute the h-part
train_unit['h [m]'] = train['Pile penetration [m]']-train['z [m]']
##compute the effective area of the pipe pile
train_unit['R_star [m]'] = pow(pow(train["Diameter [m]"],2)/4-
                          pow(train["Diameter [m]"]-2*train["Bottom wall thickness [mm]"]/1000,2)/4,0.5)
##compute max function which involves h-part
train_unit['h_R_max'] =  np.maximum(train_unit["h [m]"]/train_unit["R_star [m]"],np.full(train.shape[0],8))
##compute the unit shaft friction
train_unit['tao [MPa]'] = 0.9*0.029*1.0*train["qc [MPa]"]*pow((10.2*train["z [m]"])
                                                              /P_atm/1000,0.13)*pow(train_unit["h_R_max"],-0.38)*tan_delta
##compute the unit end bearing
train_unit['qb [MPa]'] = (0.5-0.25*np.log(train["Diameter [m]"]/D_CPT))*train["qc [MPa]"]

In [ ]:
##input data for validation set
valid_unit = pd.DataFrame()
valid_unit = valid.copy()
##compute the h-part
valid_unit['h [m]'] = valid['Pile penetration [m]']-valid['z [m]']
##compute the effective area of the pipe pile
valid_unit['R_star [m]'] = pow(pow(valid["Diameter [m]"],2)/4-
                          pow(valid["Diameter [m]"]-2*valid["Bottom wall thickness [mm]"]/1000,2)/4,0.5)
##compute max function which involves h-part
valid_unit['h_R_max'] =  np.maximum(valid_unit["h [m]"]/valid_unit["R_star [m]"],np.full(valid.shape[0],8))
##compute the unit shaft friction
valid_unit['tao [MPa]'] = 0.9*0.029*1.0*valid["qc [MPa]"]*pow((10.2*valid["z [m]"])
                                                              /P_atm/1000,0.13)*pow(valid_unit["h_R_max"],-0.38)*tan_delta
##compute the unit end bearing
valid_unit['qb [MPa]'] = (0.5-0.25*np.log(valid["Diameter [m]"]/D_CPT))*valid["qc [MPa]"]

In [ ]:
##sensitivity analysis
alpha_sen = np.linspace(1e-3,0.5,500)

MSE_train = np.zeros(len(alpha_sen))
R2_train = np.zeros(len(alpha_sen))
MSE_valid = np.zeros(len(alpha_sen))
R2_valid = np.zeros(len(alpha_sen))

t_begin = timeit.default_timer()
k = 0
for alpha in alpha_sen:
    train_unit['Rs [kN]'] = alpha*(np.pi*train_unit["Diameter [m]"]*train_unit["z [m]"].diff()*
                                   train_unit["tao [MPa]"]).cumsum()
    train_unit["Qb [kN]"] = (np.pi/4*pow(train_unit["Diameter [m]"],2))*train_unit["qb [MPa]"]
    train_unit["Qult [kN]"] = train_unit["Rs [kN]"]+train_unit["Qb [kN]"]
    
    ##linearized features
    train_unit["linearized ENTRHU"] = np.tanh(train_unit["Normalised ENTRHU [-]"]*5-0.5)
    train_unit["linearized Qult"] = np.tanh(train_unit["Qult [kN]"]/1000-1)
    train_unit["linearized z"] = np.tanh(train_unit["z [m]"]/10-0.5)
    
    x_train = train_unit.dropna()[linearized_features]
    y_train = train_unit.dropna()["Blowcount [Blows/m]"]
    
    ##fit teh model
    lm1 = LinearRegression().fit(x_train,y_train)
    y_pred_train_lm1 = lm1.predict(x_train)
    MSE_train[k] = mean_squared_error(y_train,y_pred_train_lm1)
    R2_train[k] = r2_score(y_train,y_pred_train_lm1)
    
    ##valid set
    valid_unit["Rs [kN]"] = 0.7*(np.pi*valid_unit["Diameter [m]"]*valid_unit["z [m]"].diff()*
                valid_unit["tao [MPa]"]).cumsum()
    valid_unit["Qb [kN]"] = (np.pi/4*pow(valid_unit['Diameter [m]'],2))*valid_unit["qb [MPa]"]
    valid_unit["Qult [kN]"] = valid_unit["Rs [kN]"]+valid_unit["Qb [kN]"]
    
    valid_unit["linearized ENTRHU"] = np.tanh(valid_unit["Normalised ENTRHU [-]"]*5-0.5)
    valid_unit["linearized Qult"] = np.tanh(valid_unit["Qult [kN]"]/1000-1)
    valid_unit["linearized z"] = np.tanh(valid_unit["z [m]"]/10-0.5)
    
    x_valid = valid_unit.dropna()[linearized_features]
    y_valid = valid_unit.dropna()["Blowcount [Blows/m]"]
    
    y_pred_valid_lm1 = lm1.predict(x_valid)
    
    MSE_valid[k] = mean_squared_error(y_valid,y_pred_valid_lm1)
    R2_valid[k] = r2_score(y_valid,y_pred_valid_lm1)
    
    k = k+1
t_end = timeit.default_timer()
print("Elapse time = ",t_end-t_begin)

In [ ]:
##plot the MSE and r2 score of training dataset and validation dataset
##training dataset
plt.plot(alpha_sen,MSE_train,'k')  
#plt.plot(alpha_aging_sen,MSE_aging_train_2,'b--',label = 'Valid_set2')  
plt.ylabel("MSE_train",fontsize = 16)
plt.xlabel(r"Tuning parameter $\alpha$",fontsize = 16)
plt.tick_params(labelsize = 14)
#plt.legend(loc = 'best',prop = {'size':14})
plt.show()

plt.plot(alpha_sen,R2_train,'k') 
#plt.plot(alpha_aging_sen,r2_aging_train_2,'b--',label = 'Valid_set2')   
plt.ylabel("R2_train",fontsize = 16)
plt.xlabel(r"Tuning parameter $\alpha$",fontsize = 16)
plt.tick_params(labelsize = 14)
#plt.legend(loc = 'best',prop = {'size':14})
plt.show()

##validation dataset
plt.plot(alpha_sen,MSE_valid,'k')
#plt.plot(alpha_aging_sen,MSE_aging_valid_2,'b--',label = 'Valid_set2') 
plt.ylabel("MSE_valid",fontsize = 16)
plt.xlabel(r"Tuning parameter $\alpha$",fontsize = 16)
plt.tick_params(labelsize = 14)
#plt.legend(loc = 'best',prop = {'size':14})
plt.show()
    
plt.plot(alpha_sen,R2_valid,'k')
#plt.plot(alpha_aging_sen,r2_aging_valid_2,'b--',label = 'Valid_set2')
plt.ylabel("R2_valid",fontsize = 16)
plt.xlabel(r"Tuning parameter $\alpha$",fontsize = 16)
plt.tick_params(labelsize = 14)
#plt.legend(loc = 'best',prop = {'size':14})
plt.show()

In [ ]:
print("Minimum RMSE = ",min(np.sqrt(MSE_train)))
print("Minimum RMSE = ",min(np.sqrt(MSE_valid)))
print("Maximum R^2 = ",max(R2_valid))
alpha_sen[np.argmin(MSE_valid)]

# results above show that alpha=0.014 is the optimal tuning parameter value

In [ ]:
##now set alpha = 0.014 to get the fit function (94 training sites all in)
##first define input parameters
##atmospheric pressure (MPa)
P_atm = 0.101325
##define interface friction angle between sand and pile
tan_delta = np.tan(29)
##unit of D_CPT is meter
D_CPT = 0.035814
alpha = 0.014

##input data for train set
train_unit = pd.DataFrame()
train_unit = train.copy()
##compute the h-part
train_unit['h [m]'] = train['Pile penetration [m]']-train['z [m]']
##compute the effective area of the pipe pile
train_unit['R_star [m]'] = pow(pow(train["Diameter [m]"],2)/4-
                          pow(train["Diameter [m]"]-2*train["Bottom wall thickness [mm]"]/1000,2)/4,0.5)
##compute max function which involves h-part
train_unit['h_R_max'] =  np.maximum(train_unit["h [m]"]/train_unit["R_star [m]"],np.full(train.shape[0],8))
##compute the unit shaft friction
train_unit['tao [MPa]'] = 0.9*0.029*1.0*train["qc [MPa]"]*pow((10.2*train["z [m]"])
                                                              /P_atm/1000,0.13)*pow(train_unit["h_R_max"],-0.38)*tan_delta
##compute the unit end bearing
train_unit['qb [MPa]'] = (0.5-0.25*np.log(train["Diameter [m]"]/D_CPT))*train["qc [MPa]"]

train_unit['Rs [kN]'] = alpha*(np.pi*train_unit["Diameter [m]"]*train_unit["z [m]"].diff()*
                                   train_unit["tao [MPa]"]).cumsum()
train_unit["Qb [kN]"] = (np.pi/4*pow(train_unit["Diameter [m]"],2))*train_unit["qb [MPa]"]
train_unit["Qult [kN]"] = train_unit["Rs [kN]"]+train_unit["Qb [kN]"]
    
##linearized features
train_unit["linearized ENTRHU"] = np.tanh(train_unit["Normalised ENTRHU [-]"]*5-0.5)
train_unit["linearized Qult"] = np.tanh(train_unit["Qult [kN]"]/1000-1)
train_unit["linearized z"] = np.tanh(train_unit["z [m]"]/10-0.5)
    
x_train = train_unit.dropna()[linearized_features]
y_train = train_unit.dropna()["Blowcount [Blows/m]"]
    
##fit teh model
lm1 = LinearRegression().fit(x_train,y_train)
y_pred_train_lm1 = lm1.predict(x_train)
MSE_train = mean_squared_error(y_train,y_pred_train_lm1)
R2_train = r2_score(y_train,y_pred_train_lm1)
print([MSE_train,R2_train])

In [ ]:
##get the submit file
test_unit = test.copy()
##compute the h-part
test_unit['h [m]'] = test['Pile penetration [m]']-test['z [m]']
##compute the effective area of the pipe pile
test_unit['R_star [m]'] = pow(pow(test["Diameter [m]"],2)/4-
                          pow(test["Diameter [m]"]-2*test["Bottom wall thickness [mm]"]/1000,2)/4,0.5)
##compute max function which involves h-part
test_unit['h_R_max'] =  np.maximum(test_unit["h [m]"]/test_unit["R_star [m]"],np.full(test.shape[0],8))
##compute the unit shaft friction
test_unit['tao [MPa]'] = 0.9*0.029*1.0*test["qc [MPa]"]*pow((10.2*test["z [m]"])
                                                              /P_atm/1000,0.13)*pow(test_unit["h_R_max"],-0.38)*tan_delta
##compute the unit end bearing
test_unit['qb [MPa]'] = (0.5-0.25*np.log(test["Diameter [m]"]/D_CPT))*test["qc [MPa]"]

test_unit['Rs [kN]'] = alpha*(np.pi*test_unit["Diameter [m]"]*test_unit["z [m]"].diff()*
                                   test_unit["tao [MPa]"]).cumsum()
test_unit["Qb [kN]"] = (np.pi/4*pow(test_unit["Diameter [m]"],2))*test_unit["qb [MPa]"]
test_unit["Qult [kN]"] = test_unit["Rs [kN]"]+test_unit["Qb [kN]"]
    
##linearized features
test_unit["linearized ENTRHU"] = np.tanh(test_unit["Normalised ENTRHU [-]"]*5-0.5)
test_unit["linearized Qult"] = np.tanh(test_unit["Qult [kN]"]/1000-1)
test_unit["linearized z"] = np.tanh(test_unit["z [m]"]/10-0.5)

test_unit.dropna(inplace = True)
test_unit["linearized ENTRHU"] = np.tanh(test_unit["Normalised ENTRHU [-]"]*5-0.5)
test_unit["linearized Qult"] = np.tanh(test_unit["Qult [kN]"]/1000-1)
test_unit["linearized z"] = np.tanh(test_unit["z [m]"]/10-0.5)

x_test = test_unit[linearized_features]

y_pred_lm1= lm1.predict(x_test)

test_unit['Blowcount [Blows/m]'] = y_pred_lm1

test_unit[['ID','Blowcount [Blows/m]']].to_csv("lm1_IC05_submit_0014Rs_new.csv",index = False)